## Code for reading the HAR-UML dataset

In [2]:
import numpy as np

from source.read_HAR_dataset import read_har_dataset, har_dimensions

In [3]:
# [X_train, y_train, epochs, X_test, y_test, test_epochs, X_validation, y_validation, validation_epochs, kcal_MET, test_kcal_MET, validation_kcal_MET] = 

data = read_har_dataset('./datasets/HAR-UML20/')

ids_train, X_train, y_train, I_train, train_kcal_MET = data['train']

ids_val, X_val, y_val, I_val, val_kcal_MET = data['val']

ids_test, X_test, y_test, I_test, test_kcal_MET = data['test']

IDS: [1, 2, 4, 7, 6, 3, 9, 8, 5, 0]
Train IDS: [1, 2, 4, 7, 6, 3]
Test IDS: [9, 8]
Val IDS: [5, 0]
Training files: 6 - ['dataset_S1.csv', 'dataset_S2.csv', 'dataset_S3.csv', 'dataset_S4.csv', 'dataset_S5.csv', 'dataset_S6.csv']
Train epochs: 6 - ['dataset_S1_epochs.csv', 'dataset_S2_epochs.csv', 'dataset_S3_epochs.csv', 'dataset_S4_epochs.csv', 'dataset_S5_epochs.csv', 'dataset_S6_epochs.csv']
Testing files: 2 - ['dataset_S7.csv', 'dataset_S8.csv']
Testing epochs: 2 - ['dataset_S7_epochs.csv', 'dataset_S8_epochs.csv']
Validation files: 2 - ['dataset_S9.csv', 'dataset_S10.csv']
Validation epochs: 2 - ['dataset_S9_epochs.csv', 'dataset_S10_epochs.csv']


In [10]:
# print(X_train.shape)
# y_train.shape
# I_train.shape
# train_kcal_MET[:, 1].shape

In [11]:
har_dimensions

array(['Accelerometer-X', 'Accelerometer-Y', 'Accelerometer-Z',
       'Gyrometer-X', 'Gyrometer-Y', 'Gyrometer-Z', 'Magnetometer-X',
       'Magnetometer-Y', 'Magnetometer-Z'], dtype='<U15')

In [12]:
from source.utils import create_dir, filter_dimensions, save_metrics2
dimensions = [
    'Accelerometer-X', 
    'Accelerometer-Y',
    'Accelerometer-Z',
    'Gyrometer-X',
    'Gyrometer-Y',
    'Gyrometer-Z'
]

X_train_f = filter_dimensions(X_train, har_dimensions, dimensions)
X_test_f = filter_dimensions(X_test, har_dimensions, dimensions)
X_validation_f = filter_dimensions(X_val, har_dimensions, dimensions)

In [13]:
X_train_f.shape

(5460, 200, 6)

In [14]:
import sys
sys.path.insert(0, '/home/texs/Documents/VivaBem/repositories/ts2vec')
from ts2vec import TS2Vec

In [15]:
model = TS2Vec(
    input_dims=X_train_f.shape[2],
    device=0,
    output_dims=200
)
# print(train_data)
loss_log = model.fit(
    X_train_f,
    n_epochs=20,
    verbose=True
)

Epoch #0: loss=296.8211690715331
Epoch #1: loss=11.117479090816465
Epoch #2: loss=6.225528649920243
Epoch #3: loss=5.055363068482743


KeyboardInterrupt: 

In [ ]:
train_repr = model.encode(X_train_f.astype(np.float64), encoding_window='full_series')  # n_instances x output_dims
test_repr = model.encode(X_test_f.astype(np.float64), encoding_window='full_series')  # n_instances x output_dims

In [ ]:
import umap
reducer = umap.UMAP(n_components = 2)

# reducer.fit(train_repr, y=y_train)
reducer.fit(train_repr)

In [ ]:
train_coords = reducer.transform(train_repr)
test_coords = reducer.transform(test_repr)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(train_coords[:, 0], train_coords[:, 1], c = y_train, cmap='jet')


In [ ]:
plt.scatter(test_coords[:, 0], test_coords[:, 1], c = y_test, cmap='jet')

In [ ]:
from source.classifiers import  lstm_classification

train_pred, test_pred, validation_pred =  lstm_classification(X_train_f, y_train, X_test_f, y_test, X_validation_f, y_validation, dimensions, 'har-lstm2', batch_size = 64, n_epochs=120)

In [ ]:
from source.utils import save_metrics

labels_map = {}
labels_map['lstm'] = [y_train, train_pred, y_test, test_pred, y_validation, validation_pred]

save_metrics(labels_map, "metrics_lstm.csv")

In [23]:
# test_pred[360:380]
# test_pred[360:380]
# y_test[320:340]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])